## Извлечение коллокаций

Коллокации — устойчивые n-граммы (обычно биграммы; есть множество определений, мы пока будем использовать интуитивное представление об устойчивом выражении).

Сначала давайте научимся доставать из текста биграммы и заодно фильтровать их по морфологическим характеристикам.

In [ ]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def normalize(text):
    ### YOUR CODE HERE
    return

In [ ]:
import itertools
def get_ngrams(tokens, n=2, patterns=None):
    """
    Если patterns не None, давайте проверять, что части речи биграммы есть в patterns.
    Например, patterns = ['ADJF', 'NOUN']
    Подумаем о том, хотим ли мы склеивать два слова из разных предложений.
    Можно использовать itertools.islice
    """
    ngrams = []
    ### YOUR CODE HERE
    return ngrams

Теперь протестируем на каком-нибудь тексте (давайте считать, что каждая строчка = предложение):

In [ ]:
text = """
Кречет (лат. Falco rusticolus) — птица из отряда соколообразных семейства соколиных.
Самый крупный из соколов. 
Масса самца чуть больше 1 кг, самки — до 2 кг. 
Окраска сибирского кречета светлая (светлее лапландских кречетов), но изменчивая: от буровато-серой до почти белой сверху; брюшная сторона беловатая с темным рисунком. 
Темная полоска у разреза рта («усы») почти незаметна. 
На надклювье, как у всех соколов, характерный зубец. 
Лапы жёлтые. 
Скорость в полёте высокая, после нескольких взмахов птица быстро несётся вперёд, не парит. 
Сидящий кречет держится прямо.
Кречет похож на сапсана, но крупнее и имеет относительно более длинный хвост. 
Голос также похож на голос сапсана, но грубее и ниже: хриплое «кьяк-кьяк-кьяк» или протяжное «кеек-кеек-кеек». 
Весной может издавать довольно тихую и высокую трель. 
Южный горный подвид — алтайский кречет, которого многие специалисты считают подвидом или морфой балобана, — отличается более однообразной темной окраской."""

In [ ]:
get_ngrams(normalize(text))

In [ ]:
get_ngrams(normalize(text), n=3)

In [ ]:
get_ngrams(normalize(text), patterns=['ADJF', 'NOUN'])

Теперь возьмем небольшой корпус, который лежит тут же в папке data/

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_json("data/ng_1.jsonlines", lines=True)

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
data.head(1)

Соберем биграммы из первого текста и попробуем просто найти самые частотные:

In [ ]:
from collections import Counter

In [ ]:
bigrams = get_ngrams(data['content'][0])
c = Counter(bigrams)

In [ ]:
c.most_common(10)

Можно не включать энграммы, которые содержат предлоги, союзы и т.д.
Попробуем использовать список стоп-слов:

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('russian')

def get_ngrams(tokens, n=2, patterns=None, stoplist=[]):
    ngrams = []
    ### YOUR CODE HERE
    return ngrams

В списке есть сочетания, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять частоту энграммы и поделить на сумму количеств упоминаний слов по отдельности.

In [ ]:
def scorer_simple(word_count_a, word_count_b, bigram_count, _):
    try:
        score = bigram_count / ((word_count_a + word_count_b) - bigram_count)
    except ZeroDivisionError:
        return 0
    return score

Заведем функцию, которая будет считать частоты для биграмм и слов:

In [ ]:
def collect_stats(texts, n=2):
    word_counter = Counter()
    ngram_counter = Counter()
    for text in texts:
        word_counter.update(text)
        ngram_counter.update(get_ngrams(text, 2, stoplist=stop))
    
    return word_counter, ngram_counter

И функцию, которая считает значение метрики для каждой энграммы:

In [ ]:
def score_bigrams(word_counter, bigram_counter, scorer, threshold=-100000):
    ### YOUR CODE HERE
    bigram2score = Counter()
    
    ## если метрика выше порога, добавляем в словарь
    return bigram2score

In [ ]:
texts = data['content'].apply(normalize).tolist()
word_counter, bigram_counter = collect_stats(texts)
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_simple)

In [ ]:
bigram2score.most_common(10)

Что пошло не так?

In [ ]:
def scorer_simple_smoothed(word_count_a, word_count_b, bigram_count, _, min_count=10):
    try:
        score = (bigram_count - min_count) / ((word_count_a + word_count_b) - bigram_count)
    except ZeroDivisionError:
        return 0
    return score

In [ ]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_simple_smoothed)

In [ ]:
bigram2score.most_common(15)

Уже приличнее. В [статье](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) про word2vec для склейки устойчивых словосочетаний используют такую штуку (стр. 6):

In [ ]:
def scorer_mwe(word_count_a, word_count_b, bigram_count, len_vocab, min_count=10):
    try:
        score = ((bigram_count - min_count) / (word_count_a * word_count_b)) * len_vocab
    except ZeroDivisionError:
        return 0
    return score

In [ ]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_mwe)
bigram2score.most_common(10)

Ещё одна популярная метрика - Pointwise Mutual Information (PMI, взаимная информация). 

$$PMI = \log{\frac{p(a,b)}{p(a)p(b)}}$$

Для её вычисления используются нормализованные частоты:

In [ ]:
import numpy as np
def scorer_pmi(word_count_a, word_count_b, bigram_count, _, corpus_size, minimum_count=5):
    score = (((bigram_count - minimum_count) /corpus_size) / ((word_count_a/corpus_size) * (word_count_b/corpus_size)))
    return np.log(score) / -np.log((bigram_count/corpus_size))

Придется переписать функцию, которая применяет метрику к биграммам, потому что теперь мы хотим учитывать размер корпуса, а не словаря:

In [ ]:
def score_bigrams(word_counter, bigram_counter, scorer, threshold=-100000):
    ### YOUR CODE HERE
    bigram2score = Counter()
    len_vocab = len(word_counter)
    corpus_size = sum(word_counter.values())
    for bigram in bigram_counter:
        score = scorer(word_counter[bigram[0]], word_counter[bigram[1]], 
                       bigram_counter[bigram], len_vocab, corpus_size)
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [ ]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_pmi)
bigram2score.most_common(10)

Вообще метрики для выделения коллокаций — это статистические меры/критерии ассоциации/связи. Популярная в статистике мера — [t-test](https://en.wikipedia.org/wiki/Student%27s_t-test) (он же по-русски T-критерий Стьюдента):

$$t = \frac{\bar{x} - \mu}{\sqrt{\frac{s^2}{n}}}$$

где $\bar{x}$ — наблюдаемое среднее (нормализованная частота биграммы)

$\mu$ — ожидаемое среднее (считаем, что появление каждого слова независимо, то есть произведение вероятностей)

$s$ — стандартное отклонение ($s^2$ — дисперсия; 
выбор слова описывается распределением Бернулли, поэтому $s^2 = p(1-p)$)

$n$ — размер выборки (размер корпуса)

In [ ]:
def scorer_ttest(word_count_a, word_count_b, bigram_count, _, corpus_size, minimum_count=5):
    mu = ((word_count_a/corpus_size) * (word_count_b/corpus_size))
    x_ = (bigram_count/corpus_size)
    score = (x_ - mu) / np.sqrt(x_/corpus_size)
    return score

In [ ]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_ttest)
bigram2score.most_common(10)

Есть ещё много замечательных метрик, и многие из них реализованы в модуле `nltk.collocations`

In [ ]:
import nltk
from nltk.collocations import *

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(texts)

In [ ]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

In [ ]:
finder2.nbest(bigram_measures.pmi, 20)

In [ ]:
scores = finder2.score_ngrams(bigram_measures.dice)

In [ ]:
sorted([x for x in scores if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:20]